
# 📄 Project Write-up: Hotel Inventory Forecasting & Recommendation System

### 1. What is the core business problem and why does it matter?

The core business problem is to **forecast the demand of bar inventory items in a hotel** and **recommend appropriate inventory (par) levels**. Accurate forecasts help maintain optimal stock levels, avoid stockouts or overstocking, and reduce waste. This directly affects customer satisfaction, operational efficiency, and cost savings—making it a crucial aspect of hotel operations.

---

### 2. What assumptions did you make? Why?

- Historical consumption patterns reflect future demand trends.
- Data is clean and accurate (no significant missing or corrupt values).
- External factors (seasonality, events) remain similar during prediction.
- The inventory lead time is consistent, allowing weekly aggregation.

These assumptions simplify modeling and align with real-world inventory management scenarios where past patterns often influence future needs.

---

### 3. What model did you use and why did you choose it? Why not others?

We used **Simple Moving Average and Rolling Mean** techniques to forecast future demand. These models were chosen for their:
- Interpretability and ease of deployment.
- Suitability for short-term forecasts with seasonal trends.
- Low data requirements.

More complex models (like ARIMA, XGBoost, or LSTM) could be considered if larger or more granular datasets were available, but for this project’s scope, simplicity and transparency were prioritized.

---

### 4. How does your system perform? What would you improve?

The system performs reasonably well for short-term demand estimation and helps in estimating weekly par levels. However:
- It can be improved by incorporating **seasonality detection**, **holiday effects**, and **event-based adjustments**.
- More advanced models could be explored for better accuracy.
- Alert mechanisms can be built to flag unusual patterns.

---

### 5. How would this solution work in a real hotel?

In practice:
- Forecasts would be generated weekly using updated consumption data.
- Inventory staff would use recommended par levels to place restocking orders.
- Visual dashboards or reports could assist decision-making.

---

### Optional: What would break at scale? What would you track in production?

**Scalability Issues:**
- Manual tuning of moving averages doesn’t scale across 100+ items.
- Forecasting many SKUs may need parallelization or cloud deployment.

**Production Monitoring:**
- Track forecast vs actual demand (error rates).
- Monitor outlier spikes in consumption.
- Automate retraining or model updates periodically.

---


# Hotel Inventory Forecasting & Recommendation System

This notebook predicts demand and suggests inventory (par) levels for bar items based on historical consumption data.

In [ ]:
#  1. Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

In [ ]:
#  2. Load Data
df = pd.read_csv("Consumption Dataset - Dataset.csv")
df['Date'] = pd.to_datetime(df['Date Time Served'], errors='coerce').dt.date
df = df[['Date', 'Bar Name', 'Brand Name', 'Consumed (ml)']]

In [ ]:
#  3. Group by Date, Bar, and Brand
daily = df.groupby(['Date', 'Bar Name', 'Brand Name'])['Consumed (ml)'].sum().reset_index()

In [ ]:
#  4. Focus Example: Filter for a specific bar-brand
bar_name = "Smith's Bar"
brand_name = "Grey Goose"
sample = daily[(daily['Bar Name'] == bar_name) & (daily['Brand Name'] == brand_name)].copy()

In [ ]:
#  5. Handle Missing Dates
sample = sample.sort_values('Date')
sample = sample.set_index('Date').asfreq('D', fill_value=0)
sample.index = pd.to_datetime(sample.index)

In [ ]:
#  6. Forecast Using 7-Day Moving Average
sample['7d_avg'] = sample['Consumed (ml)'].rolling(window=7).mean()
sample['7d_std'] = sample['Consumed (ml)'].rolling(window=7).std()

last_avg = sample['7d_avg'].dropna().iloc[-1] if not sample['7d_avg'].dropna().empty else 0
last_std = sample['7d_std'].dropna().iloc[-1] if not sample['7d_std'].dropna().empty else 0

In [ ]:
#  7. Forecast Next 7 Days
forecast_dates = pd.date_range(sample.index[-1] + timedelta(days=1), periods=7)
forecast = pd.DataFrame(index=forecast_dates)
forecast['Forecasted Demand (ml)'] = last_avg
forecast['Recommended Par Level (ml)'] = last_avg + 1.5 * last_std
forecast

In [ ]:
#  8. Plot Forecast
plt.figure(figsize=(10, 4))
plt.plot(forecast['Forecasted Demand (ml)'], label='Forecasted Demand')
plt.plot(forecast['Recommended Par Level (ml)'], label='Par Level (w/ buffer)')
plt.title("7-Day Forecast & Inventory Recommendation")
plt.xlabel("Date")
plt.ylabel("Volume (ml)")
plt.legend()
plt.grid()
plt.show()

## ✅ Summary
- Used 7-day moving average to forecast demand
- Added 1.5× standard deviation as buffer to determine par level
- Easy to scale to multiple bar-brand combinations
